In [3]:
openai_key = open('../open_ai.key','r').read()
import os; os.environ["OPENAI_API_KEY"] = openai_key

https://note.com/npaka/n/n716dfd26094d

## LLM生成

In [15]:
from langchain.llms import OpenAI,OpenAIChat
# OpenAIのLLMの生成
llm = OpenAI(model_name="text-davinci-003", n=2, best_of=2)
# llm = OpenAIChat()

In [16]:
# LLMの呼び出し
result = llm("ネコの鳴き声は？")
print(result)



ネコの鳴き声は「ニャー」と言われることが多いですが、実際には様々な鳴き声を出しています。例えば、「モー」や「ミャウ」、「ウー」などがあります。


In [17]:
# 高度なLLMの呼び出し
result = llm.generate(["猫の鳴き声は？", "カラスの鳴き声は？"])
print(result)

LLMResult(generations=[[Generation(text='\n\n猫の鳴き声は「ニャー」という音です。', generation_info={'finish_reason': 'stop', 'logprobs': None}), Generation(text='\n\n猫の鳴き声は「ニャー」という音です。', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nカラスの鳴き声は「カーカー」という鳴き声です。', generation_info={'finish_reason': 'stop', 'logprobs': None}), Generation(text='\n\nカラスの鳴き声は「カーカー」です。', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'total_tokens': 129, 'prompt_tokens': 26, 'completion_tokens': 103}})


In [18]:
# 出力テキスト
print("response0:", result.generations[0][0].text)
print("response1:", result.generations[1][0].text)

# 使用したトークン数
print("llm_output:", result.llm_output)

response0: 

猫の鳴き声は「ニャー」という音です。
response1: 

カラスの鳴き声は「カーカー」という鳴き声です。
llm_output: {'token_usage': {'total_tokens': 129, 'prompt_tokens': 26, 'completion_tokens': 103}}


## LLMのシリアライズ

In [26]:
from langchain.llms import OpenAI
from langchain.llms.loading import load_llm

# LLMファイルの読み込み
llm = load_llm("data/my_llm.json")

# LLMファイルの書き込み
llm.save("my_llm.json")

## LLMのキャッシュ

### インメモリー


In [27]:
import langchain
from langchain.cache import InMemoryCache

# インメモリキャッシュの準備
langchain.llm_cache = InMemoryCache()

In [28]:
%%time
llm("好きな色は？")

CPU times: user 4.17 ms, sys: 69 µs, total: 4.24 ms
Wall time: 1.78 s


'\n\n私は人工知能エージェントであり、色を好むことはできません。'

In [29]:
%%time
llm("好きな色は？")

CPU times: user 21 µs, sys: 12 µs, total: 33 µs
Wall time: 35.8 µs


'\n\n私は人工知能エージェントであり、色を好むことはできません。'

#### SQLite

In [30]:
from langchain.cache import SQLiteCache

# SQLiteキャッシュの準備
langchain.llm_cache = SQLiteCache(database_path="data/langchain.db")

## カスタムLLM

In [31]:
# パッケージのインポート
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any

# カスタムLLMの定義
class CustomLLM(LLM):
    n: int

    # LLM種別を返す
    @property
    def _llm_type(self) -> str:
        return "custom"

    # プロンプトとストップワードを受け取り応答を返す
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        return prompt[:self.n]  # 入力の最初のN文字を返す

    # IDパラメータの辞書を返す
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"n": self.n}

In [32]:
# カスタムLLMの利用
llm = CustomLLM(n=10)
llm("あいうえおかきくけこさしすせそ")

'あいうえおかきくけこ'

https://note.com/npaka/n/n97aac2da03f4

## プロンプトの機能

In [34]:
from langchain.prompts import PromptTemplate

# 複数の入力変数を持つプロンプトテンプレートの準備
multiple_input_prompt = PromptTemplate(
    input_variables=["adjective", "content"],
    template="{adjective}{content}といえば？"
)
print(multiple_input_prompt.format(adjective="かっこいい", content="動物"))

かっこいい動物といえば？


In [37]:
# !pip install jinja2

In [38]:
from langchain.prompts import PromptTemplate

# テンプレートの準備 (jinja2)
template = """
{% for item in items %}
Q: {{ item.question }}
A: {{ item.answer }}
{% endfor %}
"""

# 入力変数の準備
items=[
    {"question": "foo", "answer": "bar"},
    {"question": "1", "answer": "2"}
]

# jinja2を使ったプロンプトテンプレートの準備
jinja2_prompt = PromptTemplate(
    input_variables=["items"],
    template=template,
    template_format="jinja2"
)
print(jinja2_prompt.format(items=items))



Q: foo
A: bar

Q: 1
A: 2



In [39]:
from langchain.prompts import FewShotPromptTemplate

# 例の準備
examples = [
    {"input": "明るい", "output": "暗い"},
    {"input": "おもしろい", "output": "つまらない"},
]

# 例のプロンプト
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="入力: {input}\n出力: {output}",
)

# FewShotPromptTemplateの準備
prompt_from_string_examples = FewShotPromptTemplate(
    examples=examples, # 例
    example_prompt=example_prompt,  # 例のフォーマット
    prefix="すべての入力の反意語を与えてください",  # プレフィックス
    suffix="入力: {adjective}\n出力:",  # サフィックス
    input_variables=["adjective"],  # 入力変数
    example_separator="\n\n"  # プレフィックスと例とサフィックスを結合する文字

)
print(prompt_from_string_examples.format(adjective="大きい"))

すべての入力の反意語を与えてください

入力: 明るい
出力: 暗い

入力: おもしろい
出力: つまらない

入力: 大きい
出力:


In [40]:
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

# 例の準備
examples = [
    {"input": "明るい", "output": "暗い"},
    {"input": "おもしろい", "output": "つまらない"},
    {"input": "エネルギッシュ", "output": "無気力"},
    {"input": "高い", "output": "低い"},
    {"input": "明るい", "output": "暗い"},
    {"input": "早い", "output": "遅い"},
]

# 例のプロンプト
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="入力: {input}\n出力: {output}",
)

# LengthBasedExampleSelectorの準備
example_selector = LengthBasedExampleSelector(
    examples=examples,  # 例
    example_prompt=example_prompt,  # 例のフォーマット
    max_length=10,  # 最大長
)

# FewShotPromptTemplateの準備
prompt_from_string_examples = FewShotPromptTemplate(
    example_selector=example_selector,  # ExampleSelector
    example_prompt=example_prompt,  # 例のフォーマット
    prefix="すべての入力の反意語を与えてください",  # プレフィックス
    suffix="入力: {adjective}\n出力:",  # サフィックス
    input_variables=["adjective"],  # 入力変数
    example_separator="\n\n"  # プレフィックスと例とサフィックスを結合する文字

)
print(prompt_from_string_examples.format(adjective="大きい"))

すべての入力の反意語を与えてください

入力: 明るい
出力: 暗い

入力: おもしろい
出力: つまらない

入力: 大きい
出力:


In [41]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate

# 例の準備
examples = [
    {"input": "明るい", "output": "暗い"},
    {"input": "おもしろい", "output": "つまらない"},
    {"input": "エネルギッシュ", "output": "無気力"},
    {"input": "高い", "output": "低い"},
    {"input": "明るい", "output": "暗い"},
    {"input": "早い", "output": "遅い"},
]

# 例のプロンプト
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="入力: {input}\n出力: {output}",
)

# SemanticSimilarityExampleSelectorの準備
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,  # 例
    OpenAIEmbeddings(),  # 埋め込み生成のためのクラス
    FAISS,  # 埋め込みを保存し、類似検索するためのVectorStoreクラス
    k=1  # 作成する例の数
)

# FewShotPromptTemplateの準備
prompt_from_string_examples = FewShotPromptTemplate(
    example_selector=example_selector,  # ExampleSelector
    example_prompt=example_prompt,  # 例のフォーマット
    prefix="すべての入力の反意語を与えてください",  # プレフィックス
    suffix="入力: {adjective}\n出力:",  # サフィックス
    input_variables=["adjective"],  # 入力変数
    example_separator="\n\n"  # プレフィックス・例・サフィックスを結合する文字

)
print(prompt_from_string_examples.format(adjective="大きい"))

すべての入力の反意語を与えてください

入力: 高い
出力: 低い

入力: 大きい
出力:


https://note.com/npaka/n/n886960b89de1